# Spark RDD operation Example

## Import spark modules

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
spark = SparkSession\
        .builder\
        .appName("PythonPi")\
        .getOrCreate()

**Setup spark configuration and create a spark context**

In [3]:
sc = spark.sparkContext

## Load data from json

In [4]:
df = spark.read.json("./data/customerData.json")

**Display the header**

In [5]:
df.show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



**Print schema**

In [6]:
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



## Dataframe queries

**Show the name column**

In [7]:
df.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



**Filtering operations**

In [8]:
df.filter(df['age'] == 40).show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



**Group the data by gender and show the count**

In [9]:
df.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    4|
+------+-----+



**Group and data by dept id and carry out aggregrate operations**

In [10]:
df.groupBy("deptid").agg({"salary": "avg", "age": "max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



## Create a dataframe from a list

**Create a list of data**

In [11]:
deptList = [{'name': 'Sales','id': '100'},{'name': 'Engineering','id': '200'}]

**Create a dataframe from the list**

In [12]:
deptDF = spark.createDataFrame(deptList)

C:\Users\musar\Anaconda3\envs\tensorflow\lib\site-packages\pyspark\sql\session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


**Show the data**

In [13]:
deptDF.show()

+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



## Join and Cascade operations

In [14]:
df.join(deptDF, df.deptid == deptDF.id).show()

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 51|   100|female|    June Rutledge|  4300|100|      Sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      Sales|
| 32|   100|  male|Benjamin Garrison|  3000|100|      Sales|
| 44|   200|  male|    Nielsen Knapp|  6500|200|Engineering|
| 40|   200|  male|    Holland Drake|  4500|200|Engineering|
+---+------+------+-----------------+------+---+-----------+



In [15]:
df.filter(df["age"]>30).join(deptDF,df.deptid == deptDF.id)\
            .groupBy("deptid").agg({"salary":"avg","age":"max"}).show()

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



## Create data frames from RDD

### Method 1

**Read the line**

In [16]:
lines = sc.textFile("./data/auto-data.csv")

**Remove the first line**

In [17]:
datalines = lines.filter(lambda x: "FUELTYPE" not in x)

In [18]:
datalines.count()

197

**Split each line by comma**

In [19]:
parts = datalines.map(lambda l: l.split(","))

In [20]:
parts.take(10)

[['subaru',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '69',
  '4900',
  '31',
  '36',
  '5118'],
 ['chevrolet',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'three',
  '48',
  '5100',
  '47',
  '53',
  '5151'],
 ['mazda',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '68',
  '5000',
  '30',
  '31',
  '5195'],
 ['toyota',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '62',
  '4800',
  '35',
  '39',
  '5348'],
 ['mitsubishi',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '68',
  '5500',
  '37',
  '41',
  '5389'],
 ['honda',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '60',
  '5500',
  '38',
  '42',
  '5399'],
 ['nissan',
  'gas',
  'std',
  'two',
  'sedan',
  'fwd',
  'four',
  '69',
  '5200',
  '31',
  '37',
  '5499'],
 ['dodge',
  'gas',
  'std',
  'two',
  'hatchback',
  'fwd',
  'four',
  '68',
  '5500',
  '37',
  '41',
  '5572'],
 ['plymouth',
  'gas',
  'std',
  'two',
  'hatchback',

**Create a new RDD with reduced number of rows**

In [21]:
autoMap = parts.map(lambda p: Row(make= p[0], body=p[4], hp = int(p[7])))

In [22]:
autoMap.take(20)

[Row(body='hatchback', hp=69, make='subaru'),
 Row(body='hatchback', hp=48, make='chevrolet'),
 Row(body='hatchback', hp=68, make='mazda'),
 Row(body='hatchback', hp=62, make='toyota'),
 Row(body='hatchback', hp=68, make='mitsubishi'),
 Row(body='hatchback', hp=60, make='honda'),
 Row(body='sedan', hp=69, make='nissan'),
 Row(body='hatchback', hp=68, make='dodge'),
 Row(body='hatchback', hp=68, make='plymouth'),
 Row(body='hatchback', hp=68, make='mazda'),
 Row(body='hatchback', hp=68, make='mitsubishi'),
 Row(body='hatchback', hp=68, make='dodge'),
 Row(body='hatchback', hp=68, make='plymouth'),
 Row(body='hatchback', hp=70, make='chevrolet'),
 Row(body='hatchback', hp=62, make='toyota'),
 Row(body='hatchback', hp=68, make='dodge'),
 Row(body='hatchback', hp=58, make='honda'),
 Row(body='hatchback', hp=62, make='toyota'),
 Row(body='hatchback', hp=76, make='honda'),
 Row(body='sedan', hp=70, make='chevrolet')]

**Create dataframe from RDD**

In [23]:
autoDF = spark.createDataFrame(autoMap)

In [24]:
autoDF.show()

+---------+---+----------+
|     body| hp|      make|
+---------+---+----------+
|hatchback| 69|    subaru|
|hatchback| 48| chevrolet|
|hatchback| 68|     mazda|
|hatchback| 62|    toyota|
|hatchback| 68|mitsubishi|
|hatchback| 60|     honda|
|    sedan| 69|    nissan|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 68|     mazda|
|hatchback| 68|mitsubishi|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 70| chevrolet|
|hatchback| 62|    toyota|
|hatchback| 68|     dodge|
|hatchback| 58|     honda|
|hatchback| 62|    toyota|
|hatchback| 76|     honda|
|    sedan| 70| chevrolet|
+---------+---+----------+
only showing top 20 rows



### Method 2

**Read file using csv function**

In [26]:
autoDF1 = spark.read.csv("./data/auto-data.csv", header=True)

In [27]:
autoDF1.show()

+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|      MAKE|FUELTYPE|ASPIRE|DOORS|     BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|    subaru|     gas|   std|  two|hatchback|  fwd|     four| 69|4900|      31|     36| 5118|
| chevrolet|     gas|   std|  two|hatchback|  fwd|    three| 48|5100|      47|     53| 5151|
|     mazda|     gas|   std|  two|hatchback|  fwd|     four| 68|5000|      30|     31| 5195|
|    toyota|     gas|   std|  two|hatchback|  fwd|     four| 62|4800|      35|     39| 5348|
|mitsubishi|     gas|   std|  two|hatchback|  fwd|     four| 68|5500|      37|     41| 5389|
|     honda|     gas|   std|  two|hatchback|  fwd|     four| 60|5500|      38|     42| 5399|
|    nissan|     gas|   std|  two|    sedan|  fwd|     four| 69|5200|      31|     37| 5499|
|     dodge|     gas|   std|  two|hatchback|  fwd|     four| 68|5500| 

## Temp Tables

In [28]:
autoDF.createOrReplaceTempView("autos")

In [29]:
spark.sql("select * from autos where hp > 200").show()

+-----------+---+-------+
|       body| hp|   make|
+-----------+---+-------+
|    hardtop|207|porsche|
|    hardtop|207|porsche|
|      sedan|262| jaguar|
|convertible|207|porsche|
+-----------+---+-------+



In [30]:
df.createOrReplaceTempView("employees")

In [31]:
spark.sql("select * from employees where salary > 4000").show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
| 51|   100|female|June Rutledge|  4300|
| 44|   200|  male|Nielsen Knapp|  6500|
+---+------+------+-------------+------+



## Pandas dataframe

In [32]:
df_pandas = df.toPandas()

In [33]:
df_pandas.head()

,age,deptid,gender,name,salary
0,32,100,male,Benjamin Garrison,3000
1,40,200,male,Holland Drake,4500
2,26,100,male,Burks Velasquez,2700
3,51,100,female,June Rutledge,4300
4,44,200,male,Nielsen Knapp,6500


In [34]:
for index, row in df_pandas.iterrows():
    print(row["salary"])

3000
4500
2700
4300
6500
